In [21]:
import geopandas as gpd
import glob
import os
import csv
import numpy as np
import pandas as pd 
from datetime import datetime
import matplotlib.dates as mdates

folder_path = r"Calculations\IllegalParkinginZones"

# Find all shapefiles in the specified folder
shapefiles = glob.glob(f'{folder_path}/*.shp')

In [22]:
print("1. Converting shapefiles...")
geo_dataframes = {}
for shapefile in shapefiles:
    shapefile_name = os.path.splitext(os.path.basename(shapefile))[0]
    gdf = gpd.read_file(shapefile)
    geo_dataframes[shapefile_name] = gdf 
print("--Completed.")

1. Converting shapefiles...
--Completed.


In [23]:
print("2. Renaming shapefiles and columns...")
prefix = 's'
keys_to_remove = []
for shapefile_name in list(geo_dataframes.keys()):
    # Check if the prefix is present in the shapefile name
    if shapefile_name.startswith(prefix):
        new_shapefile_name = shapefile_name[len(prefix):] # Remove the prefix
        geo_dataframes[new_shapefile_name] = geo_dataframes.pop(shapefile_name)
    else:
        # If the key doesn't match, add it to the list of keys to remove
        keys_to_remove.append(shapefile_name)

# Remove unwanted keys
# for key in keys_to_remove:
#     del geo_dataframes[key]

# Define the column renaming dictionary
#Column renaming. ArcGIS tends to trim these longer column headings, and here we are fixing it
column_rename_dict = {
    'street_nam': 'street_name',
    'descriptio': 'description',
    'locality_n': 'locality_name',
    'utilisatio': 'utilisation'
}


# Iterate through the GeoDataFrames in the dictionary and rename columns
for shapefile_name, gdf in geo_dataframes.items():
    geo_dataframes[shapefile_name] = gdf.rename(columns=column_rename_dict)
print("--Completed.")

2. Renaming shapefiles and columns...
--Completed.


In [24]:
print("4. List of GeoDataFrames:")
for shapefile_name in geo_dataframes.keys():
    print(shapefile_name)

4. List of GeoDataFrames:
SJ_Zones_Illegal_occupancy_June22_1900_Layer
SJ_Zones_Illegal_occupancy_June22_2000_Layer
SJ_Zones_Illegal_occupancy_June22_2100_Layer
SJ_Zones_Illegal_occupancy_June22_2200_Layer
SJ_Zones_Illegal_occupancy_June23_1900_Layer
SJ_Zones_Illegal_occupancy_June23_2000_Layer
SJ_Zones_Illegal_occupancy_June23_2100_Layer
SJ_Zones_Illegal_occupancy_June23_2200_Layer


In [29]:
print("5. Assigning timestamps...") 
timestamps = {
    'June22_1900': '2024-06-22 19:00:00',
    'June22_2000': '2024-06-22 20:00:00',
    'June22_2100': '2024-06-22 21:00:00',
    'June22_2200': '2024-06-22 22:00:00',
    'June22_2300': '2024-06-22 23:00:00',
    'June23_1900': '2024-06-23 19:00:00',
    'June23_2000': '2024-06-23 20:00:00',
    'June23_2100': '2024-06-23 21:00:00',
    'June23_2200': '2024-06-23 22:00:00'
}

for shapefile_name, gdf in geo_dataframes.items():
    shapefile_name = shapefile_name.rstrip('_Layer')
    # Check if the GeoDataFrame has a specific timestamp assigned
    if shapefile_name.startswith('SJ_Zones_Illegal_') or shapefile_name.startswith('sj_occupancy'):
        timestamp_suffix = shapefile_name.split('_')[-2] + '_' + shapefile_name.split('_')[-1]
        if timestamp_suffix in timestamps:
            gdf['timestamp'] = timestamps[timestamp_suffix]
            print(f"-df {shapefile_name} was assigned timestamp: {timestamps[timestamp_suffix]}")
        else:
            print(f"No matching timestamp for {shapefile_name}.")
    else:
        print(f"GeoDataFrame {shapefile_name} does not start with 'sj_street_occupancy' or 'sj_occupancy'.")

print("--Completed.")

5. Assigning timestamps...
-df SJ_Zones_Illegal_occupancy_June22_1900 was assigned timestamp: 2024-06-22 19:00:00
-df SJ_Zones_Illegal_occupancy_June22_2000 was assigned timestamp: 2024-06-22 20:00:00
-df SJ_Zones_Illegal_occupancy_June22_2100 was assigned timestamp: 2024-06-22 21:00:00
-df SJ_Zones_Illegal_occupancy_June22_2200 was assigned timestamp: 2024-06-22 22:00:00
-df SJ_Zones_Illegal_occupancy_June23_1900 was assigned timestamp: 2024-06-23 19:00:00
-df SJ_Zones_Illegal_occupancy_June23_2000 was assigned timestamp: 2024-06-23 20:00:00
-df SJ_Zones_Illegal_occupancy_June23_2100 was assigned timestamp: 2024-06-23 21:00:00
-df SJ_Zones_Illegal_occupancy_June23_2200 was assigned timestamp: 2024-06-23 22:00:00
--Completed.


In [30]:

# geo_dataframes['Zone_sj_occupancy_June23_2200'].head()

OUTPUT_PATH = folder_path+'/CSVOutput/'

merged_df = pd.concat(geo_dataframes.values(), ignore_index=True)

merged_df.to_csv(OUTPUT_PATH+'IllegalCarsInZone.csv', index=False)



In [31]:
merged_df

,Join_Count,TARGET_FID,veh_type,Name,geometry,timestamp
0,1,1,car,Zone 1,POINT Z (457529.673 3967952.637 0.000),2024-06-22 19:00:00
1,1,2,car,Zone 1,POINT Z (457552.344 3968025.627 0.000),2024-06-22 19:00:00
2,1,3,car,Zone 1,POINT Z (457540.179 3967964.894 0.000),2024-06-22 19:00:00
3,1,4,car,Zone 1,POINT Z (457542.483 3967967.936 0.000),2024-06-22 19:00:00
4,1,5,car,Zone 1,POINT Z (457544.879 3967970.608 0.000),2024-06-22 19:00:00
...,...,...,...,...,...,...
3363,1,475,None,Zone 1,POINT Z (457747.354 3968229.874 0.000),2024-06-23 22:00:00
3364,1,476,None,Zone 1,POINT Z (457749.791 3968250.830 0.000),2024-06-23 22:00:00
3365,1,477,None,Zone 1,POINT Z (457510.368 3967952.921 0.000),2024-06-23 22:00:00
3366,1,478,None,Zone 2,POINT Z (458027.204 3968077.537 0.000),2024-06-23 22:00:00
